In [3]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_algorithms import NumPyMinimumEigensolver

In [4]:
geometry="H 0.0 0.0 0.0; H 0.0 0.0 0.74"

In [5]:
driver=PySCFDriver(
    atom=geometry,
    basis="sto-3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

In [6]:
problem=driver.run()

In [7]:
problem.num_particles

(1, 1)

In [8]:
problem.num_spatial_orbitals

2

## HF

In [9]:
hf_total_e=problem.reference_energy
nuclear_repulsion_e=problem.hamiltonian.nuclear_repulsion_energy
hf_elec_e=hf_total_e-nuclear_repulsion_e

In [10]:
print(f"hf_total: {hf_total_e}")
print(f"nuclear_repulsion: {nuclear_repulsion_e}")
print(f"hf_elec_e: {hf_elec_e}")

hf_total: -1.1167593073964255
nuclear_repulsion: 0.7151043390810812
hf_elec_e: -1.8318636464775067


## full CI

In [11]:
mapper=JordanWignerMapper()
solver=NumPyMinimumEigensolver()
alg=GroundStateEigensolver(mapper, solver)
result=alg.solve(problem=problem)

In [12]:
fci_total=result.total_energies[0]
hf_energy=problem.reference_energy
corr_energy=fci_total-hf_energy

In [13]:
print(f"fci_total: {fci_total}")
print(f"hf_energy: {hf_energy}")
print(f"corr_energy: {corr_energy}")

fci_total: -1.1372838344885032
hf_energy: -1.1167593073964255
corr_energy: -0.02052452709207775
